참고 영상
- Age and Gender Detection Using Classification and Regression | Deep Learning Project

In [2]:
from tensorflow.keras.utils import load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import seaborn as sns
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

In [36]:
Base_dir = './image_dataset'
image_classnames = os.listdir(Base_dir)
image_classnames
image_paths =[]
clothes_labels =[]
clothes_input=[]
target_image_paths=[]

for image_class in image_classnames:
    image_filenames = os.listdir(f'{Base_dir}/{image_class}')
    for image in image_filenames:
        if image == f"{image_class}_re.png":
            clothes_label = image
            #clothes_labels.append(clothes_label)
            clothes_labels.append(clothes_label)      # target
            clothes_input.append(image)               # input
            image_paths.append(f'{Base_dir}/{image_class}/{image}')
            target_image_paths.append(int(image_class))
        else:
            clothes_labels.append(clothes_label)      # target
            clothes_input.append(image)               # input
            image_paths.append(f'{Base_dir}/{image_class}/{image}')
            target_image_paths.append(int(image_class))


In [37]:
print(len(image_paths),len(clothes_input),len(clothes_labels))

171 171 171


In [38]:
import pandas as pd
df = pd.DataFrame()
df['image_path'],df['input_image'],df['target_image_path'],df['target_image'] = image_paths,clothes_input,target_image_paths,clothes_labels
df

,image_path,input_image,target_image_path,target_image
0,./image_dataset/0/0_re.png,0_re.png,0,0_re.png
1,./image_dataset/0/0_train_1.png,0_train_1.png,0,0_re.png
2,./image_dataset/0/0_train_10.png,0_train_10.png,0,0_re.png
3,./image_dataset/0/0_train_11.png,0_train_11.png,0,0_re.png
4,./image_dataset/0/0_train_12.png,0_train_12.png,0,0_re.png
...,...,...,...,...
166,./image_dataset/9/9_train_2.png,9_train_2.png,9,9_re.png
167,./image_dataset/9/9_train_3.png,9_train_3.png,9,9_re.png
168,./image_dataset/9/9_train_4.png,9_train_4.png,9,9_re.png
169,./image_dataset/9/9_train_5.png,9_train_5.png,9,9_re.png


# Exploratory Data Analysis

In [39]:
import PIL 
from PIL import Image

PIL.Image.ANTIALIAS =PIL.Image.LANCZOS
rand_index = random.randint(0, len(image_paths))

# Feature Extraction

In [40]:
def extract_image_features(images):
    features = list()

    for image in tqdm(images):
        img = load_img(image)#, grayscale=True) # loading image as numpy array
        img = img.resize((128,128), Image.ANTIALIAS) # ANTIALIAS : help in cases where there are Distortion
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 128,128,3) # considering RGB
    return features

def y_image_features(images):
    features = list()
    for image in tqdm(images):
        img = load_img(image)#, grayscale=True) # loading image as numpy array
        img = img.resize((128,128), Image.ANTIALIAS) # ANTIALIAS : help in cases where there are Distortion
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 128,128,3) # considering RGB
    return features

In [42]:
X = extract_image_features(df['image_path'])
Y = np.array(df['target_image_path'])

  0%|          | 0/171 [00:00<?, ?it/s]

In [43]:
print(X.shape)
print(Y.shape)

(171, 128, 128, 3)
(171,)


In [44]:
# Nomalizing
X = X/255.0

In [48]:
input_shape = (128,128,3)

In [49]:
inputs = Input((input_shape))
conv_1 =Conv2D(32,kernel_size=(3,3),activation='relu')(inputs)
max_1 = MaxPooling2D(pool_size=(2,2))(conv_1)
conv_2 =Conv2D(64,kernel_size=(3,3),activation='relu')(max_1)
max_2 = MaxPooling2D(pool_size=(2,2))(conv_2)
conv_3 =Conv2D(128,kernel_size=(3,3),activation='relu')(max_2)
max_3 = MaxPooling2D(pool_size=(2,2))(conv_3)
conv_4 =Conv2D(256,kernel_size=(3,3),activation='relu')(max_3)
max_4 = MaxPooling2D(pool_size=(2,1))(conv_4)

flatten = Flatten()(max_4)

#fully connected layers
dense_1 = Dense(256, activation = 'relu')(flatten)

dropout_1 = Dropout(0.3)(dense_1)

output_1 = Dense(1, activation = 'relu' , name = 'img_out')(dropout_1)
# not binary >> x sigmoid, binary_crossentropy / estimate >> relu + mae

model = Model(inputs=[inputs], outputs=[output_1])

model.compile(loss=['mae'], optimizer='adam', metrics=['accuracy'])

In [50]:
image_classifier = model.fit(x=X, y=Y, batch_size=32, epochs=50, validation_split=0.2)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 634ms/step - accuracy: 0.0578 - loss: 4.6480 - val_accuracy: 0.0000e+00 - val_loss: 4.3394
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 582ms/step - accuracy: 0.0322 - loss: 3.2706 - val_accuracy: 0.0000e+00 - val_loss: 5.4592
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 637ms/step - accuracy: 0.0322 - loss: 3.4070 - val_accuracy: 0.0000e+00 - val_loss: 5.3323
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 512ms/step - accuracy: 0.0413 - loss: 3.1425 - val_accuracy: 0.0000e+00 - val_loss: 4.6530
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 531ms/step - accuracy: 0.0379 - loss: 3.0156 - val_accuracy: 0.0000e+00 - val_loss: 5.4917
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 535ms/step - accuracy: 0.0457 - loss: 3.5014 - val_accuracy: 0.0000e+00 - val_loss: 5.0472
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 684ms/step - accuracy: 0.0509 - loss: 3.2600 - val_accuracy: 0.0000e+00 - val_loss: 4.2856
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 547ms/step - accuracy: 0.0240 - loss: 3.0989 - val_